In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)


from aiida.orm.code import Code

from aiida.orm.data.base import Int, Float, Str

from aiida.work.run import run, async, submit, restart

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview

import numpy as np
from collections import OrderedDict
width = "150px"

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData

from apps.nanoribbon.workchain import NanoribbonWorkChain

import ipywidgets as ipw
from IPython.display import display
import nglview

In [ ]:
# query AiiDA database for structures
qb = QueryBuilder()
qb.append(StructureData)
   
#print qb.count()

all_nodes = [ node_tuple[0] for node_tuple in qb.iterall() ]
all_nodes.sort(key=lambda n: n.pk)

all_structs = OrderedDict()
all_structs["Select a Structure"] = False

print("Loading structures...")
for node in all_nodes:
    label = "%s PK: %d %s"%(node.get_formula(), node.pk, node.description)
    all_structs[label] = node

def on_struct_change(c):
    global atoms
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = drop_struct.value
    if not node:
        return
    
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

drop_struct = ipw.Dropdown(options=all_structs, value=False)
drop_struct.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([drop_struct, viewer]))

In [ ]:
qb = QueryBuilder()
qb.append(Code)
all_codes = list(qb.iterall())

# query AiiDA database for Codes
qb = QueryBuilder()
qb.append(Code)
   
all_nodes = [ node_tuple[0] for node_tuple in qb.iterall() ]
all_nodes.sort(key=lambda n: n.pk)

all_codes = OrderedDict()

for node in all_nodes:
    label = "%s PK: %d"%(node.label, node.pk)
    all_codes[label] = node

default_pw_code = [v for k, v in all_codes.items() if k.lower().startswith("pw")][-1]
default_pp_code = [v for k, v in all_codes.items() if k.lower().startswith("pp")][-1]
drop_pw_code = ipw.Dropdown(description="PW Code", options=all_codes, value=default_pw_code)
drop_pp_code = ipw.Dropdown(description="PP Code", options=all_codes, value=default_pp_code)
slider_precision = ipw.FloatSlider(description='Precision:', value=1.0, min=0.1, max=3.0, step=0.1)
display(ipw.VBox([drop_pw_code, drop_pp_code, slider_precision]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        pw_code = drop_pw_code.value
        pp_code = drop_pp_code.value
        struct = drop_struct.value
        precision = Float(slider_precision.value)

        if struct == None:
            print("Please select a structure")
            return

        outputs = submit(NanoribbonWorkChain,
                         pw_code=pw_code,
                         pp_code=pp_code,
                         structure=struct,
                         precision=precision)
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)